In [1]:
from concurrent.futures import ThreadPoolExecutor
import os
import cv2
import concurrent.futures
import mediapipe as mp
import json
from pathlib import Path

In [2]:
src_folder = "./dataset/origin"
dest_folder = "./dataset/plus"

def process_folder(folder_name):
    folder_path = os.path.join(src_folder, folder_name)
    if os.path.isdir(folder_path):
        dest_folder_path = os.path.join(dest_folder, folder_name)
        os.makedirs(dest_folder_path, exist_ok=True)
        for filename in os.listdir(folder_path):
            if filename.endswith(".jpg"):
                img_path = os.path.join(folder_path, filename)
                img = cv2.imread(img_path)
                flipped_img = cv2.flip(img, 1)
                right_img_name = f"{folder_name}_right_{filename}"
                right_img_path = os.path.join(dest_folder_path, right_img_name)
                cv2.imwrite(right_img_path, img)
                left_img_name = f"{folder_name}_left_{filename}"
                left_img_path = os.path.join(dest_folder_path, left_img_name)
                cv2.imwrite(left_img_path, flipped_img)
                
if not os.path.exists(dest_folder):
    os.makedirs(dest_folder)
    num_threads = os.cpu_count() or 1
    with ThreadPoolExecutor(max_workers=num_threads) as executor:
        folder_names = os.listdir(src_folder)
        executor.map(process_folder, folder_names)
else:
    print("The destination folder already exists. Skipping processing.")


In [3]:
def normalize_bbox(image_width, image_height, bbox):
    x, y, width, height = bbox
    return x / image_width, y / image_height, width / image_width, height / image_height

def detect_and_draw_hand(image_path, labels, output_image_folder):
    mp_hands = mp.solutions.hands
    image = cv2.imread(image_path)
    image_rgb = cv2.cvtColor(image, cv2.COLOR_BGR2RGB)
    image_height, image_width, _ = image.shape

    with mp_hands.Hands(static_image_mode=True, max_num_hands=1) as hands:
        results = hands.process(image_rgb)
        normalized_bbox = {}
        file_name = Path(image_path).stem

        if results.multi_hand_landmarks:
            hand_landmarks = results.multi_hand_landmarks[0]
            x_min, y_min = image_width, image_height
            x_max, y_max = 0, 0

            for landmark in hand_landmarks.landmark:
                x, y = int(landmark.x * image_width), int(landmark.y * image_height)
                x_min, y_min = min(x, x_min), min(y, y_min)
                x_max, y_max = max(x, x_max), max(y, y_max)

            padding = 0.1
            x_min = max(0, int(x_min - (x_max - x_min) * padding))
            y_min = max(0, int(y_min - (y_max - y_min) * padding))
            x_max = min(image_width, int(x_max + (x_max - x_min) * padding))
            y_max = min(image_height, int(y_max + (y_max - y_min) * padding))

            normalized_bbox = normalize_bbox(image_width, image_height, (x_min, y_min, x_max - x_min, y_max - y_min))
            cv2.rectangle(image, (x_min, y_min), (x_max, y_max), (0, 255, 0), 2)

            path = f"{output_image_folder}/{labels}"
            os.makedirs(path, exist_ok=True)
            cv2.imwrite(f"{path}/{file_name}.jpg", image)

        return {
            "bboxes": [normalized_bbox],
            "labels": [labels],
            "leading_hand": "Left" if 'left' in file_name else "Right"
        }

def process_image(filename, folder_path, labels, output_image_folder):
    if filename.endswith(".jpg") or filename.endswith(".png"):
        image_path = os.path.join(folder_path, filename)
        output_data = detect_and_draw_hand(image_path, labels, output_image_folder)
        return Path(filename).stem, output_data

src_folder = "./dataset/plus"
output_json_folder = "./dataset/json"
output_image_folder = "./dataset/mark"

if not os.path.exists(output_image_folder) or not os.path.exists(output_json_folder):
    os.makedirs(output_image_folder, exist_ok=True)
    os.makedirs(output_json_folder, exist_ok=True)
    for folder_name in os.listdir(src_folder):
        folder_path = os.path.join(src_folder, folder_name)
        labels = os.path.basename(folder_path)
        output_json = f"{output_json_folder}/{labels}.json"
        output_json_data = {}

        with ThreadPoolExecutor() as executor:
            futures = [executor.submit(process_image, filename, folder_path, labels, output_image_folder) for filename in os.listdir(folder_path)]
            for future in concurrent.futures.as_completed(futures):
                result = future.result()
                if result is not None:
                    stem, output_data = result
                    output_json_data[stem] = output_data

        with open(output_json, "w") as json_file:
            json.dump(output_json_data, json_file, indent=4)
else:
    print("Output folders already exist. Skipping processing.")


FileNotFoundError: [Errno 2] No such file or directory: './dataset/json/A.json'

In [ ]:

def remove_unmatched_files(dataset_plus_path, dataset_mark_path):
    for root, dirs, files in os.walk(dataset_plus_path):
        for file in files:
            plus_file_path = os.path.join(root, file)
            mark_file_path = plus_file_path.replace(dataset_plus_path, dataset_mark_path)
            if not os.path.exists(mark_file_path):
                print("Removing:", plus_file_path)
                os.remove(plus_file_path)
dataset_plus_path = "./dataset/plus"
dataset_mark_path = "./dataset/mark"


remove_unmatched_files(dataset_plus_path, dataset_mark_path)
